# Incremental DFA Decomposition: Per-Step Work Scaling

This notebook demonstrates that the incremental `TruncatedIncrementalDFADecomp`
does per-step work proportional to the **change** in the DFA graph (dirty + border
states), not the total DFA size or target string length.

We run several FSTs through long `>>` chains and show:
1. **Total DFA states** grows linearly with target length.
2. **States expanded per step** stays constant (after initial transient).
3. **Wall-clock time per step** stays constant, while from-scratch construction grows.

### How the measurement works

Each `TruncatedIncrementalDFADecomp` instance records `_stats` with:
- `n_dirty` / `n_border`: states invalidated by the target extension
- `n_expanded`: states whose arcs were (re-)computed in the BFS
- `n_arcs`: arcs added during expansion
- `total_dfa_states`: cumulative DFA states after this step
- `n_frontier`: states that will be dirty on the *next* extension

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from transduction import examples
from transduction.dfa_decomp_incremental_truncated import TruncatedIncrementalDFADecomp

In [ ]:
def collect_incremental_stats(fst, target_string):
    """Run incremental >> chain, collecting per-step stats and timing."""
    records = []
    s = TruncatedIncrementalDFADecomp(fst, '')
    for i, y in enumerate(target_string):
        t0 = time.perf_counter()
        s = s >> y
        dt = time.perf_counter() - t0
        rec = dict(s._stats)
        rec['step'] = i + 1
        rec['time_s'] = dt
        records.append(rec)
    return records


def collect_from_scratch_stats(fst, target_string):
    """Build from scratch for each target prefix, collecting timing."""
    records = []
    for i in range(1, len(target_string) + 1):
        prefix = target_string[:i]
        t0 = time.perf_counter()
        s = TruncatedIncrementalDFADecomp(fst, prefix)
        dt = time.perf_counter() - t0
        rec = dict(s._stats)
        rec['step'] = i
        rec['time_s'] = dt
        records.append(rec)
    return records


def rolling_median(data, window=7):
    out = []
    for i in range(len(data)):
        lo = max(0, i - window // 2)
        hi = min(len(data), i + window // 2 + 1)
        out.append(np.median(data[lo:hi]))
    return out

In [ ]:
N = 200

benchmarks = [
    ('triplets of doom', examples.triplets_of_doom(), 'a' * N),
    ('3-tuples of doom', examples.doom(set('abc'), 3), ('abc' * 80)[:N]),
    ('lookahead',        examples.lookahead(),         ('ab' * 120)[:N]),
    ('duplicate',        examples.duplicate(set('12345')), ('1122334455' * 25)[:N]),
]

results = {}
for name, fst, target in benchmarks:
    incr = collect_incremental_stats(fst, target)
    scratch = collect_from_scratch_stats(fst, target)
    results[name] = {'incremental': incr, 'from_scratch': scratch}
    print(f'{name}: {len(target)} steps, final DFA size = {incr[-1]["total_dfa_states"]}')

## Total DFA states vs. states expanded per step

The left axis (blue) shows the total DFA size growing linearly with target length.
The right axis (red) shows the per-step work (states expanded) staying **flat**.

This is the main claim: per-step cost is O(|change|), not O(|full DFA|).

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Total DFA size (blue, left axis) vs. states expanded per step (red, right axis)', fontsize=13)

for ax, name in zip(axes.flat, results):
    incr = results[name]['incremental']
    steps = [r['step'] for r in incr]
    total = [r['total_dfa_states'] for r in incr]
    expanded = [r['n_expanded'] for r in incr]

    ax2 = ax.twinx()
    ax.plot(steps, total, 'b-', linewidth=1.5, label='total DFA states')
    ax2.plot(steps, expanded, 'r-', linewidth=1.5, label='states expanded')

    ax.set_xlabel('target length')
    ax.set_ylabel('total DFA states', color='b')
    ax2.set_ylabel('states expanded (per step)', color='r')
    ax.set_title(name)
    ax.tick_params(axis='y', labelcolor='b')
    ax2.tick_params(axis='y', labelcolor='r')
    ax2.set_ylim(bottom=0, top=max(expanded) * 2.5)

plt.tight_layout()
#plt.savefig('reports/incremental_scaling_states.png', dpi=150, bbox_inches='tight')
plt.show()

## Breakdown: dirty, border, expanded, frontier, arcs

All five per-step quantities stabilize after an initial transient (steps 1-2)
and remain constant regardless of how large the DFA grows.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Per-step work breakdown (all quantities stay bounded)', fontsize=13)

for ax, name in zip(axes.flat, results):
    incr = results[name]['incremental']
    steps = [r['step'] for r in incr]

    for key, color, label in [
        ('n_dirty',    'tab:red',    'dirty'),
        ('n_border',   'tab:orange', 'border'),
        ('n_expanded', 'tab:blue',   'expanded'),
        ('n_frontier', 'tab:green',  'frontier'),
        ('n_arcs',     'tab:purple', 'arcs added'),
    ]:
        ax.plot(steps, [r[key] for r in incr], color=color, label=label, linewidth=1.2)

    ax.set_xlabel('target length')
    ax.set_ylabel('count')
    ax.set_title(name)
    ax.legend(fontsize=7, loc='upper right')
    ax.set_ylim(bottom=0)

plt.tight_layout()
#plt.savefig('reports/incremental_scaling_breakdown.png', dpi=150, bbox_inches='tight')
plt.show()

## Wall-clock time: incremental vs. from-scratch

From-scratch rebuilds the full DFA each step (O(|total states|) BFS).
Incremental `>>` only re-expands dirty+border states (O(|change|)).

Rolling median smooths sub-millisecond timer noise.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Wall-clock time per step: incremental (blue) vs from-scratch (gray)\n(rolling median, window=7)', fontsize=13)

for ax, name in zip(axes.flat, results):
    incr = results[name]['incremental']
    scratch = results[name]['from_scratch']
    steps_i = [r['step'] for r in incr]
    steps_s = [r['step'] for r in scratch]
    t_incr = [r['time_s'] * 1000 for r in incr]
    t_scratch = [r['time_s'] * 1000 for r in scratch]

    ax.plot(steps_s, rolling_median(t_scratch),
            color='gray', linewidth=1.5, alpha=0.8, label='from scratch')
    ax.plot(steps_i, rolling_median(t_incr),
            color='tab:blue', linewidth=1.5, label='incremental >>')

    ax.set_xlabel('target length')
    ax.set_ylabel('time (ms)')
    ax.set_title(name)
    ax.legend(fontsize=8)
    ax.set_ylim(bottom=0)

plt.tight_layout()
#plt.savefig('reports/incremental_scaling_time.png', dpi=150, bbox_inches='tight')
plt.show()

## Summary table

Steady-state per-step metrics (median over steps 6+) vs final DFA size.

In [ ]:
print(f'{"FST":25s} {"final states":>12s} {"expanded/step":>14s} {"dirty/step":>11s} {"border/step":>12s} {"incr (ms)":>10s} {"scratch (ms)":>12s} {"speedup":>8s}')
print('-' * 105)
for name in results:
    incr = results[name]['incremental']
    scratch = results[name]['from_scratch']
    steady = incr[5:]
    steady_s = scratch[5:]
    final_total = incr[-1]['total_dfa_states']
    med_expanded = np.median([r['n_expanded'] for r in steady])
    med_dirty = np.median([r['n_dirty'] for r in steady])
    med_border = np.median([r['n_border'] for r in steady])
    med_time = np.median([r['time_s'] for r in steady]) * 1000
    med_time_s = np.median([r['time_s'] for r in steady_s]) * 1000
    speedup = med_time_s / med_time if med_time > 0 else float('inf')
    print(f'{name:25s} {final_total:12d} {med_expanded:14.0f} {med_dirty:11.0f} {med_border:12.0f} {med_time:10.3f} {med_time_s:12.3f} {speedup:8.1f}x')

## Direct correlation: time vs. change size

The left panel sweeps 15 FSTs (`doom(V, K)` with varying `|V|` and `K`, plus
`triplets_of_doom`, `lookahead`, `duplicate`) to get a range of per-step change
sizes (3–40 states expanded). Each point is one FST's steady-state median.
Time is proportional to change size (R² = 0.97).

The right panel shows all 15 FSTs' raw per-step timings vs the total DFA size
at that step. Each FST traces a horizontal band: as its DFA grows from tens to
thousands of states, per-step time stays flat. The vertical position of each band
is determined by the FST's change size, not the DFA size.

In [ ]:
def collect_with_reps(fst, target_string, n_reps=5):
    """Collect per-step stats with repeated timing for robust medians."""
    records = []
    for rep in range(n_reps):
        s = TruncatedIncrementalDFADecomp(fst, '')
        for i, y in enumerate(target_string):
            t0 = time.perf_counter()
            s = s >> y
            dt = time.perf_counter() - t0
            if rep == 0:
                rec = dict(s._stats)
                rec['step'] = i + 1
                rec['times'] = [dt]
                records.append(rec)
            else:
                records[i]['times'].append(dt)
    for rec in records:
        rec['time_s'] = np.median(rec['times'])
    return records

# Sweep doom(V, K) with varying |V| and K to get a range of change sizes
M = 150
sweep_fsts = []
for k in [2, 3]:
    for asize in [2, 3, 4, 5, 6, 8]:
        alpha = [chr(ord('a') + i) for i in range(asize)]
        V = set(alpha)
        target = (''.join(alpha) * (M // asize + 1))[:M]
        sweep_fsts.append((f'doom(|V|={asize}, K={k})', examples.doom(V, k), target))
sweep_fsts += [
    ('triplets of doom',  examples.triplets_of_doom(), 'a' * M),
    ('lookahead',         examples.lookahead(),        ('ab' * 80)[:M]),
    ('duplicate(5)',      examples.duplicate(set('12345')), ('1122334455' * 20)[:M]),
]

sweep_data = {}
for name, fst, target in sweep_fsts:
    sweep_data[name] = collect_with_reps(fst, target, n_reps=5)

# --- Two-panel figure ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5.5))

# LEFT: Across FSTs — steady-state time vs steady-state change size
ss_expanded, ss_time, ss_labels = [], [], []
for name, recs in sweep_data.items():
    steady = recs[5:]
    ss_expanded.append(np.median([r['n_expanded'] for r in steady]))
    ss_time.append(np.median([r['time_s'] for r in steady]) * 1e6)
    ss_labels.append(name)

ss_expanded = np.array(ss_expanded)
ss_time = np.array(ss_time)

ax1.scatter(ss_expanded, ss_time, s=50, zorder=5, color='tab:blue', edgecolors='black', linewidths=0.5)
slope = np.dot(ss_expanded, ss_time) / np.dot(ss_expanded, ss_expanded)
xs = np.linspace(0, ss_expanded.max() * 1.05, 100)
ax1.plot(xs, slope * xs, 'r--', linewidth=1.5, label=f'fit: {slope:.1f} μs/state')
ss_res = np.sum((ss_time - slope * ss_expanded) ** 2)
ss_tot = np.sum((ss_time - ss_time.mean()) ** 2)
r2 = 1 - ss_res / ss_tot
ax1.set_xlabel('states expanded per step (steady-state median)', fontsize=11)
ax1.set_ylabel('time per step (μs, steady-state median)', fontsize=11)
ax1.set_title(f'Across FSTs: time vs. change size\nR² = {r2:.2f}', fontsize=12)
ax1.legend(fontsize=9)
ax1.set_xlim(left=0); ax1.set_ylim(bottom=0)
for i, name in enumerate(ss_labels):
    if any(tag in name for tag in ['triplet', 'lookahead', 'duplicate', '|V|=8']):
        ax1.annotate(name, (ss_expanded[i], ss_time[i]),
                     textcoords='offset points', xytext=(6, 4), fontsize=6.5, color='gray')

# RIGHT: Within each FST — time vs total DFA size (should be flat)
cmap = plt.cm.tab20
for idx, (name, recs) in enumerate(sweep_data.items()):
    steady = recs[5:]
    totals = [r['total_dfa_states'] for r in steady]
    times_us = [r['time_s'] * 1e6 for r in steady]
    color = cmap(idx / len(sweep_data))
    ax2.plot(totals, times_us, '.', color=color, markersize=3, alpha=0.4)
    if len(times_us) > 10:
        med = rolling_median(times_us, window=7)
        label = name if idx < 8 else None
        ax2.plot(totals, med, '-', color=color, linewidth=1.2, label=label)

ax2.set_xlabel('total DFA states (growing over >> chain)', fontsize=11)
ax2.set_ylabel('time per step (μs)', fontsize=11)
ax2.set_title('Within each FST: time stays flat\nas DFA grows', fontsize=12)
ax2.legend(fontsize=6.5, loc='upper left', ncol=2)
ax2.set_ylim(bottom=0)

plt.tight_layout()
#plt.savefig('reports/incremental_scaling_correlation.png', dpi=150, bbox_inches='tight')
plt.show()